In [ ]:
%%script false --no-raise-error
%%capture
# Install pip packages in the current Jupyter kernel
# %%script false --no-raise-error (above) disables the cell.
# %%capture (above) suppresses output.
import sys
!{sys.executable} -m pip install poetry
!{sys.executable} -m pip install jupyterlab-widgets
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import codecs, os
from tempfile import NamedTemporaryFile

import ipywidgets
from IPython.display import display

from parsing.string_to_program import string_to_program
from parsing.string_to_ltl import string_to_ltl
from programs.synthesis.synthesis import synthesize, setup_synthesis, compute_abstraction
# Sanity check that we are in the right folder
print(os.getcwd())

In [ ]:
EXAMPLES = {
'parallel/arbiter': ("examples/parallel/arbiter/program.prog", "examples/parallel/arbiter/controller.tlsf"),
'parallel/arbiter2': ("examples/parallel/arbiter/program2.prog", "examples/parallel/arbiter/controller.tlsf")
}

example_or_upload = ipywidgets.Dropdown(
    options=[*EXAMPLES.keys(), 'None, upload files'],
    value='parallel/arbiter',
    # description='',
    disabled=False,
)
prog_uploader = ipywidgets.FileUpload(
    accept='.prog',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False,
    description="Upload Program file (.prog)",
    layout={'width': 'max-content'}
)
tlsf_uploader = ipywidgets.FileUpload(
    accept='.tlsf',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False,
    description="Upload specification file (.tlsf)",
    layout={'width': 'max-content'}
)
upload_box = ipywidgets.HBox([prog_uploader, tlsf_uploader])

# display(example_or_upload)
stack_list = [ipywidgets.Label("")] * len(EXAMPLES)
stack_list.append(upload_box)
stack = ipywidgets.Stack(stack_list, selected_index=0)
ipywidgets.jslink((example_or_upload, 'index'), (stack, 'selected_index'))
vbox = ipywidgets.VBox([example_or_upload, stack])
display(ipywidgets.HTML("<h2>Choose an example or upload input files</h2>"))
display(vbox)

In [ ]:
if example_or_upload.value not in EXAMPLES:
    if (prog_uploader.value == () or tlsf_uploader.value == ()):
        display(ipywidgets.HTML("AAAAA"))
    else:
        date_file = codecs.decode(prog_uploader.value[0].content, encoding="utf8")
        with NamedTemporaryFile('wb', suffix='.tlsf', delete=False) as tmp_tlsf:
            tmp_tlsf.write(tlsf_uploader.value[0].content)
            tlsf_path = tmp_tlsf.name
else:
    program_path, tlsf_path = EXAMPLES.get(example_or_upload.value, (None, None))
    date_file = open(program_path, "r").read()

In [ ]:
# Run this only once (or re-run to reset the synthesis loop)

date = string_to_program(date_file)
ltl_assumptions, ltl_guarantees, in_acts, out_acts = setup_synthesis(date, None, tlsf_path)
state_predicates, transition_predicates = [], []

In [ ]:
step = compute_abstraction(date, ltl_assumptions, ltl_guarantees, in_acts, out_acts, '', state_predicates, transition_predicates, notebook=True)
step[1].to_dot(step[2])